In [86]:
from __future__ import print_function
import os.path
import httplib2
#import apiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import datetime
import pandas as pd


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

def set_row_displays(n):
    pd.set_option('display.max_columns', None)  # or 1000
    pd.set_option('display.max_rows', n)  # or 1000
    pd.set_option('display.max_colwidth', None)  # or 199

def get_df(data):
    data_dict = {}
    cols = data[0]
    for col in cols:
        data_dict[col] = []
    width = len(cols)
    for i in range(len(cols)):
        for k in range(1, len(data)):
            
            if len(data[k]) == width:
                data_dict[cols[i]].append(data[k][i])
    
        
    data_df = pd.DataFrame.from_dict(data_dict)
    return data_df


"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret_852918827851-jphlh48nvfk0rairmigc9vm4kbaaeeeb.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='managers').execute()
managers = result.get('values', [])
managers_df = get_df(managers)

result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='leads').execute()
leads = result.get('values', [])
leads_df = get_df(leads)

result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='transactions').execute()
transactions = result.get('values', [])
transactions_df = get_df(transactions)

result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='clients').execute()
clients = result.get('values', [])
clients_df = get_df(clients)


In [87]:
clients_df[clients_df['client_id'] == '00000000-0000-0000-0000-000000000000']

,client_id,created_at,l_manager_id


In [88]:
set_row_displays(5)
leads_df['created_at'] = leads_df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
leads_df

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
...,...,...,...,...,...,...
3335,ffc0b363-2ae9-11eb-ac1f-c412f533dba1,2020-11-20 08:36:48,cpc,instagram,ad52c7a8-a752-11e7-8115-c412f533dba1,00000000-0000-0000-0000-000000000000
3336,ffcdebc8-15c2-11eb-ac1d-c412f533dba1,2020-10-24 10:34:43,utm_medium,utm_source,ad52c7a8-a752-11e7-8115-c412f533dba1,f3357d13-15c7-11eb-ac1d-c412f533dba1


In [89]:
set_row_displays(50)
leads_df.groupby('l_manager_id').count()


,lead_id,created_at,d_utm_medium,d_utm_source,l_client_id
l_manager_id,,,,,
00000000-0000-0000-0000-000000000000,491,491,491,491,491
0c5d545d-06ff-11eb-ac18-c412f533dba1,1,1,1,1,1
0d789282-f8ae-11ea-ac18-c412f533dba1,2,2,2,2,2
1bb0e7ca-5892-11ea-abeb-c412f533dba1,1,1,1,1,1
1e9c5562-0cfc-11ea-abe1-c412f533dba1,280,280,280,280,280
2346423e-adcf-11e2-921f-002522b9dae9,1,1,1,1,1
31298620-0355-11eb-ac18-c412f533dba1,1,1,1,1,1
31d3f905-0c7c-11eb-ac1a-c412f533dba1,1,1,1,1,1
43756fa4-57a0-11e9-ab9a-c412f533dba1,341,341,341,341,341


In [90]:
managers_df = managers_df.rename(columns={"manager_id": 'l_manager_id'})
managers_df

,l_manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,43756fa4-57a0-11e9-ab9a-c412f533dba1,manager #11,club #4
2,543888c4-fbb3-11e9-abdc-c412f533dba1,manager #12,club #3
3,5dd88f32-6501-11ea-abf3-c412f533dba1,manager #10,club #2
4,9a87c608-52dc-11ea-abeb-c412f533dba1,manager #9,club #3
5,abed5496-ca88-11e9-abb8-c412f533dba1,manager #5,club #4
6,ad52c7a8-a752-11e7-8115-c412f533dba1,manager #7,club #1
7,b55a233c-5df6-11ea-abf2-c412f533dba1,manager #2,club #4
8,b832aac3-fcda-11ea-ac18-c412f533dba1,manager #13,club #3
9,d12fe765-b444-11e9-abb3-c412f533dba1,manager #1,club #1


In [91]:
leads_with_clubs = pd.merge(leads_df, managers_df, how='inner', on=['l_manager_id'])

In [92]:
set_row_displays(10)
leads_with_clubs

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2
1,0127f039-0541-11eb-ac18-c412f533dba1,2020-10-03 10:23:52,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,b7b746d1-1adf-11e8-812f-c412f533dba1,manager #8,club #2
2,02554de5-12c9-11eb-ac1c-c412f533dba1,2020-10-20 15:40:11,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,214a6764-d5ae-11e9-abbe-c412f533dba1,manager #8,club #2
3,028dc5b3-1852-11eb-ac1d-c412f533dba1,2020-10-27 16:43:28,cpc,insta,1e9c5562-0cfc-11ea-abe1-c412f533dba1,50010dbc-03c2-11ea-abdf-c412f533dba1,manager #8,club #2
4,02c07025-2f34-11eb-ac1f-c412f533dba1,2020-11-25 19:36:46,,,1e9c5562-0cfc-11ea-abe1-c412f533dba1,02c07026-2f34-11eb-ac1f-c412f533dba1,manager #8,club #2
...,...,...,...,...,...,...,...,...
2766,f9fd43ed-28a4-11eb-ac1f-c412f533dba1,2020-11-17 11:17:40,utm_medium,utm_source,abed5496-ca88-11e9-abb8-c412f533dba1,c06cf2de-6fd6-11e7-80fc-c412f533dba1,manager #5,club #4
2767,fa010347-0f8c-11eb-ac1a-c412f533dba1,2020-10-16 12:52:56,,,abed5496-ca88-11e9-abb8-c412f533dba1,00000000-0000-0000-0000-000000000000,manager #5,club #4
2768,fc8075bb-1acf-11eb-ac1d-c412f533dba1,2020-10-30 20:50:17,,,abed5496-ca88-11e9-abb8-c412f533dba1,fc8075bc-1acf-11eb-ac1d-c412f533dba1,manager #5,club #4
2769,fe8be0d9-2690-11eb-ac1d-c412f533dba1,2020-11-14 19:49:36,cpc,yandex,abed5496-ca88-11e9-abb8-c412f533dba1,15fdbdc4-2713-11eb-ac1d-c412f533dba1,manager #5,club #4


In [93]:
channels = leads_with_clubs['d_utm_source'].unique()
print(channels)
managers = leads_with_clubs['d_manager'].unique()
print(manager)
clubs = leads_with_clubs['d_club'].unique()
print(club)

['viber' 'insta' '' 'instagram' 'yandex' 'utm_source' 'vk' 'vkontakte'
 'sms' 'ycard' 'google' 'ycard#!/tproduct/225696739-1498486363994' 'ig']
['manager #8' 'manager #1' 'manager #11' 'manager #10' 'manager #2'
 'manager #4' 'manager #9' 'manager #13' 'manager #7' 'manager #12'
 'manager #5' 'manager #3']
club #3


In [94]:
set_row_displays(200)
leads_with_clubs.groupby(['d_utm_source', 'd_manager', 'd_club']).count()

lead_id  \
d_utm_source                             d_manager   d_club             
                                         manager #1  club #1       51   
                                         manager #10 club #2       59   
                                         manager #11 club #4      101   
                                         manager #12 club #3       42   
                                         manager #13 club #3       92   
                                         manager #2  club #4       61   
                                         manager #4  club #2       93   
                                         manager #5  club #4       90   
                                         manager #7  club #1       59   
                                         manager #8  club #2       77   
                                         manager #9  club #3       62   
google                                   manager #1  club #1       21   
                                         manager #10 club #2       14   
                                         manager #11 club #4       21   
                                         manager #12 club #3        7   
                                         manager #13 club #3       15   
                                         manager #2  club #4        7   
                                         manager #4  club #2        9   
                                         manager #5  club #4       11   
                                         manager #7  club #1        7   
                                         manager #8  club #2       13   
                                         manager #9  club #3       13   
ig                                       manager #10 club #2        1   
                                         manager #11 club #4        2   
insta                                    manager #1  club #1        4   
                                         manager #10 club #2        1   
                                         manager #11 club #4        1   
                                         manager #12 club #3        1   
                                         manager #13 club #3        1   
                                         manager #4  club #2        3   
                                         manager #5  club #4        1   
                                         manager #8  club #2        2   
                                         manager #9  club #3        5   
instagram                                manager #1  club #1      146   
                                         manager #10 club #2       39   
                                         manager #11 club #4      109   
                                         manager #12 club #3       23   
                                         manager #13 club #3      108   
                                         manager #2  club #4       75   
                                         manager #3  club #1        1   
                                         manager #4  club #2       68   
                                         manager #5  club #4       74   
                                         manager #7  club #1       80   
                                         manager #8  club #2       77   
                                         manager #9  club #3       83   
sms                                      manager #1  club #1        7   
                                         manager #10 club #2       11   
                                         manager #11 club #4        6   
                                         manager #12 club #3        5   
                                         manager #13 club #3        7   
                                         manager #2  club #4       13   
                                         manager #4  club #2       12   
                                         manager #5  club #4        4   
                                         manage

In [95]:
for ch in channels:
    for man in managers:
        for club in clubs:
            print(ch, man, club)
            print(leads_with_clubs[leads_with_clubs['d_utm_source'] == ch][leads_with_clubs['d_manager'] == man][leads_with_clubs['d_club'] == club])

viber manager #8 club #2
                                  lead_id          created_at d_utm_medium  \
0    00678e71-0fb9-11eb-ac1a-c412f533dba1 2020-10-16 18:08:09          sms   
1    0127f039-0541-11eb-ac18-c412f533dba1 2020-10-03 10:23:52          sms   
2    02554de5-12c9-11eb-ac1c-c412f533dba1 2020-10-20 15:40:11          sms   
5    02df5b28-12aa-11eb-ac1c-c412f533dba1 2020-10-20 11:58:20          sms   
44   202ea177-0ecf-11eb-ac1a-c412f533dba1 2020-10-15 14:13:53          sms   
45   21c4a77f-03e8-11eb-ac18-c412f533dba1 2020-10-01 17:15:10          sms   
58   329ff313-12d1-11eb-ac1c-c412f533dba1 2020-10-20 16:38:57          sms   
66   389bb173-0946-11eb-ac1a-c412f533dba1 2020-10-08 13:11:18          sms   
74   424fe908-0947-11eb-ac1a-c412f533dba1 2020-10-08 13:18:43          sms   
102  58ac867e-12d1-11eb-ac1c-c412f533dba1 2020-10-20 16:39:51          sms   
117  64c49de7-1f64-11eb-ac1d-c412f533dba1 2020-11-05 16:42:42          cpi   
173  9cbe3e2b-0403-11eb-ac18-c412f533db

/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match Dat

                                   lead_id          created_at d_utm_medium  \
1674  02f7a793-12ca-11eb-ac1c-c412f533dba1 2020-10-20 15:47:21          sms   
1701  1bc9d46f-1440-11eb-ac1d-c412f533dba1 2020-10-22 12:25:15          sms   
1709  25782c4c-1440-11eb-ac1d-c412f533dba1 2020-10-22 12:25:31          sms   
1722  3609f755-12c8-11eb-ac1c-c412f533dba1 2020-10-20 15:34:28          sms   
1731  45a59480-12c8-11eb-ac1c-c412f533dba1 2020-10-20 15:34:58          sms   
1738  4b6ccb09-1357-11eb-ac1c-c412f533dba1 2020-10-21 08:38:42          sms   
1741  4c86f02a-0951-11eb-ac1a-c412f533dba1 2020-10-08 14:30:35          sms   
1743  4f23cca4-094c-11eb-ac1a-c412f533dba1 2020-10-08 13:54:55          sms   
1744  51471d47-12c8-11eb-ac1c-c412f533dba1 2020-10-20 15:35:14          sms   
1745  52b81292-0ef6-11eb-ac1a-c412f533dba1 2020-10-15 18:54:28          sms   
1757  5dbdcdab-1901-11eb-ac1d-c412f533dba1 2020-10-28 13:38:43          sms   
1783  7882f9c2-12ca-11eb-ac1c-c412f533dba1 2020-10-2

/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match Dat

 club #4
                                  lead_id          created_at d_utm_medium  \
632  065cc0b2-1535-11eb-ac1d-c412f533dba1 2020-10-23 17:38:25       target   

    d_utm_source                          l_manager_id  \
632        insta  43756fa4-57a0-11e9-ab9a-c412f533dba1   

                              l_client_id    d_manager   d_club  
632  25defe7d-fce1-11ea-ac18-c412f533dba1  manager #11  club #4  
insta manager #11 club #3
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
insta manager #10 club #2
                                   lead_id          created_at d_utm_medium  \
1056  73d8304a-1a93-11eb-ac1d-c412f533dba1 2020-10-30 13:36:57       social   

     d_utm_source                          l_manager_id  \
1056        insta  5dd88f32-6501-11ea-abf3-c412f533dba1   

                               l_client_id    d_manager   d_club  
1056  698e7a6d-d5b3-11e9-abbe-c412f533dba1  manager #10  

...

/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match Dat


ycard#!/tproduct/225696739-1498486363994 manager #5 club #4
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ycard#!/tproduct/225696739-1498486363994 manager #5 club #3
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ycard#!/tproduct/225696739-1498486363994 manager #3 club #2
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ycard#!/tproduct/225696739-1498486363994 manager #3 club #1
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ycard#!/tproduct/225696739-1498486363994 manager #3 club #4
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ycard#!/tproduct/225696739-1498486363994 man

/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match Dat

Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #13 club #3
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #7 club #2
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #7 club #1
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #7 club #4
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #7 club #3
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id, d_manager, d_club]
Index: []
ig manager #12 club #2
Empty DataFrame
Columns: [lead_id, created_at, d_utm_medium, d_utm_source, l_man

/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match Dat

In [96]:
class SpecifiedReport:
    
    def __init__(self, channel, manager, club, df):
        uncreated_client_id = '00000000-0000-0000-0000-000000000000'
        
        self.channel = channel
        self.manager = manager
        self.club = club
        self.df = df[df['d_utm_source'] == channel][df['d_manager'] == manager][df['d_club'] == club]
        
        self.n_trash_leads = self.df[self.df['l_client_id'] == uncreated_client_id].shape[0]
        self.n_leads = self.df.shape[0]
        
        self.df = self.df[self.df['l_client_id'] != uncreated_client_id]
        
    df = None
    channel = ""
    manager = ""
    club = ""
    
    n_leads = 0
    n_trash_leads = 0
    n_new_leads = 0
    n_customers = 0
    n_new_customers = 0
    new_customers_purchases_amount = 0
    

In [101]:
r = SpecifiedReport("viber", "manager #8", "club #2", leads_with_clubs)
set_row_displays(30)
r.df[["created_at", "l_client_id"]].sort_values(by=['created_at']).drop_duplicates(subset='l_client_id', keep='last')



/home/dmitri/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,created_at,l_client_id
45,2020-10-01 17:15:10,e7dadac5-171e-11e8-812f-c412f533dba1
173,2020-10-01 20:31:53,76c2734b-b3f1-11e7-8116-c412f533dba1
197,2020-10-02 13:07:30,1dc95406-f16d-11e8-ab82-c412f533dba1
1,2020-10-03 10:23:52,b7b746d1-1adf-11e8-812f-c412f533dba1
226,2020-10-08 13:15:36,7a007fc1-0f32-11e8-812b-c412f533dba1
74,2020-10-08 13:18:43,c8532a34-6fd7-11e7-80fc-c412f533dba1
44,2020-10-15 14:13:53,10530fbf-ed99-11e9-abda-c412f533dba1
0,2020-10-16 18:08:09,7264cf21-a31d-11e9-abb0-c412f533dba1
5,2020-10-20 11:58:20,8520501c-f5e8-11e7-8126-c412f533dba1
253,2020-10-20 12:11:51,5ca794a0-0a83-11e9-ab86-c412f533dba1


In [98]:
r.df

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2
1,0127f039-0541-11eb-ac18-c412f533dba1,2020-10-03 10:23:52,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,b7b746d1-1adf-11e8-812f-c412f533dba1,manager #8,club #2
2,02554de5-12c9-11eb-ac1c-c412f533dba1,2020-10-20 15:40:11,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,214a6764-d5ae-11e9-abbe-c412f533dba1,manager #8,club #2
5,02df5b28-12aa-11eb-ac1c-c412f533dba1,2020-10-20 11:58:20,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,8520501c-f5e8-11e7-8126-c412f533dba1,manager #8,club #2
44,202ea177-0ecf-11eb-ac1a-c412f533dba1,2020-10-15 14:13:53,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,10530fbf-ed99-11e9-abda-c412f533dba1,manager #8,club #2
45,21c4a77f-03e8-11eb-ac18-c412f533dba1,2020-10-01 17:15:10,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,e7dadac5-171e-11e8-812f-c412f533dba1,manager #8,club #2
58,329ff313-12d1-11eb-ac1c-c412f533dba1,2020-10-20 16:38:57,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,e6822529-4f36-11ea-abeb-c412f533dba1,manager #8,club #2
66,389bb173-0946-11eb-ac1a-c412f533dba1,2020-10-08 13:11:18,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,721854e8-47fc-11ea-abeb-c412f533dba1,manager #8,club #2
74,424fe908-0947-11eb-ac1a-c412f533dba1,2020-10-08 13:18:43,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,c8532a34-6fd7-11e7-80fc-c412f533dba1,manager #8,club #2
102,58ac867e-12d1-11eb-ac1c-c412f533dba1,2020-10-20 16:39:51,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,721854e8-47fc-11ea-abeb-c412f533dba1,manager #8,club #2
